#### Read local _align.nc file and plot of 60 hz biolume raw data against other variables

This Notebook uses Holoviz & Bokeh to plot time series data from the biolume sensor and *does* work inside VS Code.

The full resolution data in the _align.nc files is written by xarray with int64 nanosecond timestamps. We like this as integer values avoid roundoff errors that can occur with floating point values. Unfortunately netCDF3 and Hyrax on dods.mbari.org do not support int64 values. So we have to read the data into xarry directly from the local netCDF4 file. There is commented out code in the third cell to read from a test hyrax server should you want to try it.

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join("../src/data"))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
from logs2netcdfs import BASE_PATH, MISSIONNETCDFS

# Assumes that data have been processed locally using src/data/process_dorado.py
auv_name = "dorado"

In [ ]:
# Pick mission
missions_dir = os.path.join(BASE_PATH, auv_name, MISSIONNETCDFS)
mission_picker = widgets.Dropdown(
    options=[f for f in sorted(os.listdir(missions_dir)) if f != ".DS_Store"]
)
display(mission_picker)

In [ ]:
# Load full resolution data from the _align.nc file into an xarray Dataset
netcdfs_dir = os.path.join(BASE_PATH, auv_name, MISSIONNETCDFS, mission_picker.value)
opendap_base = os.path.join("http://dods.mbari.org:8181/opendap/data/auvctd/surveys/", mission_picker.value.split('.')[0], 'netcdf')
# Use `opendap_base` to test new hyrax-docker opendap server that supports int64 data types
ds_align = xr.open_dataset(os.path.join(netcdfs_dir, f"{auv_name}_{mission_picker.value}_align.nc"))
#ds_1S = xr.open_dataset(os.path.join(opendap_base, f"{auv_name}_{mission_picker.value}_1S.nc"))
print("New hyrax-docker link: " + os.path.join(opendap_base, f"{auv_name}_{mission_picker.value}_align.nc"))
ds_align


In [ ]:
# After reading the data, we can plot what we want by re-executing this cell
# Do not commit to the repo the output of this cell, it is too large!
hv.extension("bokeh")   # See https://github.com/holoviz/holoviews/issues/4861#issuecomment-1239739743
biolume_raw_plot = ds_align["biolume_raw"].to_pandas().rename("raw").hvplot()
#biolume_max_1S_plot = ds_align["biolume_raw"].to_pandas().resample('1S').max().rename("max_1S").hvplot()
biolume_min_plot = ds_align["biolume_raw"].rolling(**{"biolume_time60hz": 300}).min().to_pandas().rename("min").hvplot()
#biolume_min_sm_plot = ds_align["biolume_raw"].rolling(**{"biolume_time60hz": 300}).min().rolling(**{"biolume_time60hz": 300}, center=True).min().to_pandas().rename("sm").hvplot()
biolume_avg_plot = ds_align["biolume_avg_biolume"].to_pandas().rename("avg").hvplot()
#biolume_avg_1S_plot = ds_1S["biolume_avg_biolume"].to_pandas().rename("avg_1S").hvplot()
#biolume_raw_1S_plot = ds_1S["biolume_raw"].to_pandas().rename("raw_1S").hvplot()
#biolume_raw_plot * biolume_avg_plot * biolume_max_1S_plot * biolume_raw_1S_plot
biolume_raw_plot * biolume_avg_plot * biolume_min_plot